In [1]:
import re
import time
import requests
import pandas as pd
from io import StringIO
from bs4 import BeautifulSoup
from fuzzymatcher import link_table, fuzzy_left_join

In [13]:

import fresh_data.get_datasets
import importlib
importlib.reload(fresh_data.get_datasets)
from fresh_data.get_datasets import *

In [14]:
polarization = load_polarization_data()
polarization.head()

,representative,congress,chamber,icpsr,state_icpsr,district_code,state_abbrev,party_code,last_means,bioguide_id,...,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,nokken_poole_dim1,nokken_poole_dim2,state_name,year_range
32557,"DICKINSON, William Louis",101,House,10717,41,2,AL,200,1.0,D000326,...,0.398,-0.057,-257.54155,0.696,710.0,106.0,0.402,-0.246,Alabama,1989-1991
32558,"BEVILL, Tom",101,House,11000,41,4,AL,100,1.0,B000431,...,-0.213,0.976,-172.45593,0.789,726.0,76.0,-0.182,0.853,Alabama,1989-1991
32559,"NICHOLS, William Flynt",101,House,11037,41,3,AL,100,1.0,N000095,...,-0.042,0.872,NaN,NaN,0.0,NaN,NaN,NaN,Alabama,1989-1991
32560,"FLIPPO, Ronnie Gene",101,House,14419,41,5,AL,100,1.0,F000208,...,-0.175,0.834,-125.82210,0.804,576.0,59.0,-0.138,0.698,Alabama,1989-1991
32561,"ERDREICH, Ben",101,House,15022,41,6,AL,100,1.0,E000201,...,-0.060,0.372,-214.47508,0.752,751.0,101.0,-0.085,0.352,Alabama,1989-1991


In [4]:
polarization["congress"].unique()

array([101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
       114, 115, 116])

In [5]:
year = 1989
polarization["year_range"] = polarization["congress"].apply(lambda x: f"{ year+((x-101)*2) }-{ year+((x-101)*2)+2 }")

for congress in polarization["congress"].unique():
    print(congress, polarization[polarization["congress"]==congress].iloc[0]["year_range"])

101 1989-1991
102 1991-1993
103 1993-1995
104 1995-1997
105 1997-1999
106 1999-2001
107 2001-2003
108 2003-2005
109 2005-2007
110 2007-2009
111 2009-2011
112 2011-2013
113 2013-2015
114 2015-2017
115 2017-2019
116 2019-2021


In [6]:
polarization.columns

Index(['representative', 'congress', 'chamber', 'icpsr', 'state_icpsr',
       'district_code', 'state_abbrev', 'party_code', 'last_means',
       'bioguide_id', 'born', 'died', 'nominate_dim1', 'nominate_dim2',
       'nominate_log_likelihood', 'nominate_geo_mean_probability',
       'nominate_number_of_votes', 'nominate_number_of_errors',
       'nokken_poole_dim1', 'nokken_poole_dim2', 'state_name', 'year_range'],
      dtype='object')

In [7]:
polarization.congress.max()

116

In [8]:
polarization.state_name.unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
       'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype=object)

In [9]:
polarization[polarization["congress"]==106]

,representative,congress,chamber,icpsr,state_icpsr,district_code,state_abbrev,party_code,last_means,bioguide_id,...,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,nokken_poole_dim1,nokken_poole_dim2,state_name,year_range
34784,"CALLAHAN, Herbert Leon (Sonny)",106,House,15090,41,1,AL,200,1.0,C000052,...,0.373,0.202,-156.97696,0.83000,842.0,61.0,0.290,0.263,Alabama,1999-2001
34785,"CRAMER, Robert E. (Bud), Jr.",106,House,29100,41,5,AL,100,1.0,C000868,...,-0.132,0.612,-241.95350,0.75500,861.0,102.0,-0.124,0.540,Alabama,1999-2001
34786,"EVERETT, Robert Terry",106,House,29300,41,2,AL,200,1.0,E000268,...,0.414,0.528,-120.12012,0.86600,834.0,43.0,0.447,0.563,Alabama,1999-2001
34787,"BACHUS, Spencer T., III",106,House,29301,41,6,AL,200,1.0,B000013,...,0.387,0.228,-197.21883,0.79600,863.0,83.0,0.305,0.103,Alabama,1999-2001
34788,"HILLIARD, Earl Frederick",106,House,29302,41,7,AL,100,1.0,H000621,...,-0.555,0.569,-217.22676,0.77300,845.0,83.0,-0.575,0.651,Alabama,1999-2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35219,"RYAN, Paul D.",106,House,29939,25,1,WI,200,1.0,R000570,...,0.556,-0.250,-229.70399,0.76772,869.0,98.0,0.527,-0.287,Wisconsin,1999-2001
35220,"BALDWIN, Tammy",106,House,29940,25,2,WI,100,1.0,B001230,...,-0.492,-0.134,-186.93015,0.80133,844.0,91.0,-0.526,-0.414,Wisconsin,1999-2001
35221,"GREEN, Mark",106,House,29941,25,8,WI,200,1.0,G000545,...,0.456,-0.294,-226.45812,0.76900,864.0,102.0,0.452,-0.220,Wisconsin,1999-2001
35222,"BARRETT, Thomas M.",106,House,39315,25,5,WI,100,1.0,B000177,...,-0.380,-0.507,-241.57378,0.75700,868.0,113.0,-0.451,-0.519,Wisconsin,1999-2001


In [10]:
steven_mask = polarization["representative"].apply(lambda x: "LaTOURETTE".lower() in x.lower())
polarization[steven_mask].sort_values("congress", ascending=False)

,representative,congress,chamber,icpsr,state_icpsr,district_code,state_abbrev,party_code,last_means,bioguide_id,...,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,nokken_poole_dim1,nokken_poole_dim2,state_name,year_range
37787,"LaTOURETTE, Steven C.",112,House,29553,24,14,OH,200,1.0,L000553,...,0.211,-0.063,-316.31448,0.795,1382.0,116.0,0.179,0.073,Ohio,2011-2013
37339,"LaTOURETTE, Steven C.",111,House,29553,24,14,OH,200,1.0,L000553,...,0.211,-0.063,-236.94223,0.781,959.0,97.0,0.197,0.106,Ohio,2009-2011
36884,"LaTOURETTE, Steven C.",110,House,29553,24,14,OH,200,1.0,L000553,...,0.211,-0.063,-363.34834,0.766,1361.0,146.0,0.223,-0.082,Ohio,2007-2009
36431,"LaTOURETTE, Steven C.",109,House,29553,24,14,OH,200,1.0,L000553,...,0.211,-0.063,-206.33184,0.787,862.0,73.0,0.228,-0.092,Ohio,2005-2007
35988,"LaTOURETTE, Steven C.",108,House,29553,24,14,OH,200,1.0,L000553,...,0.211,-0.063,-150.77440,0.831,815.0,44.0,0.214,-0.025,Ohio,2003-2005
35543,"LaTOURETTE, Steven C.",107,House,29553,24,19,OH,200,1.0,L000553,...,0.211,-0.063,-112.67914,0.842,654.0,40.0,0.213,-0.023,Ohio,2001-2003
35102,"LaTOURETTE, Steven C.",106,House,29553,24,19,OH,200,1.0,L000553,...,0.211,-0.063,-205.52357,0.784,844.0,87.0,0.216,-0.188,Ohio,1999-2001
34661,"LaTOURETTE, Steven C.",105,House,29553,24,19,OH,200,1.0,L000553,...,0.211,-0.063,-232.86269,0.777,923.0,99.0,0.213,-0.251,Ohio,1997-1999
34218,"LaTOURETTE, Steven C.",104,House,29553,24,19,OH,200,1.0,L000553,...,0.211,-0.063,-232.38541,0.818,1158.0,90.0,0.227,-0.153,Ohio,1995-1997


In [11]:
rep_mask = polarization["representative"].apply(lambda rep: "Conor".lower() in rep.lower())
polarization[rep_mask]

,representative,congress,chamber,icpsr,state_icpsr,district_code,state_abbrev,party_code,last_means,bioguide_id,...,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,nokken_poole_dim1,nokken_poole_dim2,state_name,year_range
39143,"LAMB, Conor",115,House,21756,14,18,PA,100,NaN,L000588,...,-0.143,0.212,-73.25637,0.75848,265.0,40.0,-0.09,0.021,Pennsylvania,2017-2019
39590,"LAMB, Conor",116,House,21756,14,17,PA,100,NaN,L000588,...,-0.143,0.212,-119.40512,0.86231,806.0,61.0,-0.14,0.297,Pennsylvania,2019-2021


In [12]:
rep_mask = fec["representative"].apply(lambda rep: "ISAKSON".lower() in rep.lower() or ("Enrico".lower() in rep.lower()))
fec[rep_mask]

NameError: name 'fec' is not defined

In [ ]:
def fuzzy_entity_res(rep_1, rep_2):
    """
        Returns an integer prediction of how close two strings are in similarity.
        100 is the highest level of similarity. 0 is the lowest.
    """
    if pd.isna(rep_2):
        return 0
    prediction = fuzz.partial_ratio(rep_1.strip().lower(),rep_2.strip().lower())
    return prediction

def check_subset(row_1, df_2, suffix_1, suffix_2):
    """
        Perform entity resolution on a record in the polarize and census df 
        Only parses a subset of the FEC df which has matches in state, and district
        Then uses The Fuzz(TM) to find the best match within the subset.
    """

    # get subset of df_2 with matches in state and district:
    match_subset = (df_2["district_code"] == row_1["district_code"]) & (df_2["state_name"] == row_1["state_name"])
    df_2_subset = df_2[match_subset]
    
    if len(df_2_subset) == 0:
        print(f"ERROR: no matches in df_2 for district: '{row_1['district_code']}', state: '{row_1['state_name']}'")
        return row_1

    df_2_subset["distance"] = df_2_subset.apply(lambda row_2: fuzzy_entity_res(row_1[f"representative_{suffix_1}"], row_2[f"representative_{suffix_2}"]), axis=1)

    # display(df_2_subset)

    closest_match_row = df_2_subset[df_2_subset["distance"]==df_2_subset["distance"].max()].iloc[0] # Get closest match
    row_1[f"{suffix_1}-{suffix_2}_closeness"] = df_2_subset['distance'].max()
    if df_2_subset["distance"].max() < 70: # YOUNG, Donald Edwin - YOUNG, DONALD E is scored as 73, this should be a match.
        print(f"no match, closest: {df_2_subset['distance'].max()}, for {suffix_1}: {row_1[f'representative_{suffix_1}']} - {suffix_2}: {closest_match_row[f'representative_{suffix_2}']}")
        print("congress:", row_1["congress"], "| district:", row_1["district_code"])
        # Instantiate the columns in row_1 with the columns from df_2:
        for column in df_2.columns:
            row_1[column] = np.nan
        print(row_1["representative"])
        return row_1
    else: # if we have a match above 70, replace instantiated values with values from row_2
        # print(f"match, on closest: {df_2_subset['distance'].max()}, for {row_1[f'representative_{suffix_1}']} - {closest_match_row[f'representative_{suffix_2}']}")
        for column in df_2_subset.columns:
            row_1[column] = closest_match_row[column]
        return row_1
        
def fuzzy_merge(df_1, df_2, suffix_1, suffix_2):
    # Apply merge algorithm on each record of df_1
    df_1.loc[:, f"representative_{suffix_1}"] = df_1["representative"]
    df_2.loc[:, f"representative_{suffix_2}"] = df_2["representative"]

    # Only include matches, remove all failed matches (NaNs):
    match_df = df_1.apply(lambda row_1: check_subset(row_1, df_2, suffix_1, suffix_2), axis=1)
    # return match_df[~pd.isna(match_df["representative"])]
    return match_df

# fec_116 = fec[fec["congress"]==116]
# polarization_116 = polarization[polarization["congress"]==116]
# test = fuzzy_merge(polarization_116, fec, "polarization", "fec")

test = fuzzy_merge(polarization, fec, "polarization", "fec")
test

NameError: name 'fec' is not defined